## Top200 SpotifyCharts


Proceso:
 - Sacar top200 de 2017, 2018, 2019 y 2010
 - De canciones duplicadas sumar los Streams y unir, quitar duplicados
 - Juntar Dataframes de años y añadir una nueva columna con el mismo.


## Preparación

In [1]:
# Help: https://github.com/kelvingakuo/fycharts#in
#! pip install fycharts

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from datetime import date

#Fechas 2020
fecha_fin_2020 = '2020-05-13' # Empieza a levantarse restricciones de salida
fecha_inicial_2020 = '2020-03-14' #14 Marzo se decreta el estado de alarma, con 5753 casos y 136 fallecidos
fecha_fin2_2020 = datetime.strptime(fecha_fin_2020, '%Y-%m-%d')
fecha_inicial2_2020 = datetime.strptime(fecha_inicial_2020, '%Y-%m-%d')

#Fechas 2019
fecha_fin_2019 = '2019-05-13'
fecha_inicial_2019 = '2019-03-14'
fecha_fin2_2019 = datetime.strptime(fecha_fin_2019, '%Y-%m-%d')
fecha_inicial2_2019 = datetime.strptime(fecha_inicial_2019, '%Y-%m-%d')

print('Días totales seleccionados 2020:', fecha_fin2_2020 - fecha_inicial2_2020)
print('Días totales seleccionados 2019:', fecha_fin2_2019 - fecha_inicial2_2019)

Días totales seleccionados 2020: 60 days, 0:00:00
Días totales seleccionados 2019: 60 days, 0:00:00


In [11]:
# ¡IMPORTANTE! Ejecutar una única vez para descargar los datos, configurado para España
'''
# Descarga año 2020
from fycharts.SpotifyCharts import SpotifyCharts
import sqlalchemy

api = SpotifyCharts()
connector = sqlalchemy.create_engine("sqlite:///spotifycharts_2020.db", echo=False)
api.top200Daily(output_file = "top_200_daily_2020.csv", output_db = connector, webhook = "https://mywebhookssite.com/post/", 
                start = fecha_inicial_2020, end = fecha_fin_2020, region = ["es"])


# Descarga año 2019
from fycharts.SpotifyCharts import SpotifyCharts
import sqlalchemy

api = SpotifyCharts()
connector = sqlalchemy.create_engine("sqlite:///spotifycharts_2019.db", echo=False)
api.top200Daily(output_file = "top_200_daily_2019.csv", output_db = connector, webhook = "https://mywebhookssite.com/post/", 
                start = fecha_inicial_2019, end = fecha_fin_2019, region = ["es"])
'''

INFO : 12/07/2020 05:09:08 PM : Extracting top 200 daily for 2020-03-14 - es
INFO : 12/07/2020 05:09:09 PM : Extracting top 200 daily for 2020-03-15 - es
INFO : 12/07/2020 05:09:09 PM : Appending data to the table top_200_daily
INFO : 12/07/2020 05:09:09 PM : POSTing data to the endpoint h
INFO : 12/07/2020 05:09:09 PM : Appending data to the file top_200_daily_2020.csv...
Exception in thread Thread-18:
Traceback (most recent call last):
  File "/Users/victormac/anaconda3/lib/python3.7/site-packages/fycharts/SpotifyCharts.py", line 87, in __post_to_endpoint_from_queue
    postToRestEndpoint(df, url, what_data)
  File "/Users/victormac/anaconda3/lib/python3.7/site-packages/fycharts/write_to_outputs.py", line 54, in postToRestEndpoint
    raise(e)
  File "/Users/victormac/anaconda3/lib/python3.7/site-packages/fycharts/write_to_outputs.py", line 51, in postToRestEndpoint
    requests.post(url, json = dump)
  File "/Users/victormac/anaconda3/lib/python3.7/site-packages/requests/api.py", li

## Importar la tabla

In [2]:
# Importamos la tabla con los datos y miramos a ver qué tal están
data_top200_2020= pd.read_csv("top_200_daily_2020.csv")
data_top200_2019= pd.read_csv("top_200_daily_2019.csv")
print(data_top200_2020.isnull().sum())
print('2020:', data_top200_2020.shape)
print('2019:',data_top200_2019.shape)
data_top200_2020.head()

Position      0
Track Name    0
Artist        0
Streams       0
date          0
region        0
spotify_id    0
dtype: int64
2020: (12200, 7)
2019: (12200, 7)


,Position,Track Name,Artist,Streams,date,region,spotify_id
0,1,Tusa,KAROL G,446086,2020-03-14,es,7k4t7uLgtOxPwTpFmtJNTY
1,2,La Difícil,Bad Bunny,399788,2020-03-14,es,6NfrH0ANGmgBXyxgV2PeXt
2,3,Tattoo,Rauw Alejandro,380503,2020-03-14,es,7na7Bk98usp84FaOJFPv3d
3,4,Diosa,Myke Towers,336910,2020-03-14,es,3JHpk0DOTOzyh0777JFAky
4,5,Rojo,J Balvin,327388,2020-03-14,es,4uziEsK1yiqdauKVDPsmVG


In [3]:
# Importamos con las columnas que nos interesan
data_top200_2020 = pd.read_csv("top_200_daily_2020.csv", parse_dates= ["date"], usecols=['Position','Track Name', 'Artist', 'date', 'Streams', 'date', 'spotify_id'])
data_top200_2019 = pd.read_csv("top_200_daily_2019.csv", parse_dates= ["date"], usecols=['Position','Track Name', 'Artist', 'date', 'Streams', 'date', 'spotify_id'])
#Meto el año en nueva columnas columnas
data_top200_2020['year'] = pd.DatetimeIndex(data_top200_2020['date']).year
data_top200_2019['year'] = pd.DatetimeIndex(data_top200_2019['date']).year

#Concateno los datos de los dos años
data_top200 = pd.concat([data_top200_2020, data_top200_2019], axis=0,)

print(data_top200.dtypes)
print(data_top200.nunique()) # Ojo, debería haber el mismo número de Track Names y spotify ID!!
print(data_top200.shape)

Position               int64
Track Name            object
Artist                object
Streams                int64
date          datetime64[ns]
spotify_id            object
year                   int64
dtype: object
Position        200
Track Name      738
Artist          297
Streams       20942
date            122
spotify_id      807
year              2
dtype: int64
(24400, 7)


In [4]:
data_top200.loc[data_top200['spotify_id'] == '6NfrH0ANGmgBXyxgV2PeXt']

,Position,Track Name,Artist,Streams,date,spotify_id,year
1,2,La Difícil,Bad Bunny,399788,2020-03-14,6NfrH0ANGmgBXyxgV2PeXt,2020
201,2,La Difícil,Bad Bunny,372705,2020-03-15,6NfrH0ANGmgBXyxgV2PeXt,2020
401,2,La Difícil,Bad Bunny,389384,2020-03-16,6NfrH0ANGmgBXyxgV2PeXt,2020
601,2,La Difícil,Bad Bunny,390067,2020-03-17,6NfrH0ANGmgBXyxgV2PeXt,2020
801,2,La Difícil,Bad Bunny,392738,2020-03-18,6NfrH0ANGmgBXyxgV2PeXt,2020
...,...,...,...,...,...,...,...
11217,18,La Difícil,Bad Bunny,165292,2020-05-09,6NfrH0ANGmgBXyxgV2PeXt,2020
11426,27,La Difícil,Bad Bunny,151313,2020-05-10,6NfrH0ANGmgBXyxgV2PeXt,2020
11628,29,La Difícil,Bad Bunny,166240,2020-05-11,6NfrH0ANGmgBXyxgV2PeXt,2020
11828,29,La Difícil,Bad Bunny,143671,2020-05-12,6NfrH0ANGmgBXyxgV2PeXt,2020


In [7]:
# Agrupar sumas y convertir en Dataframe
data_top200_sumastreams = data_top200.groupby('spotify_id')['Streams'].sum()
data_top200_sumastreams = pd.DataFrame({'spotify_id':data_top200_sumastreams.index, 'Streams':data_top200_sumastreams.values})
print(data_top200_sumastreams.shape)
data_top200_sumastreams.head()

(807, 2)


,spotify_id,Streams
0,00ZBADBKZGwnzGIAA6U9Fb,44107
1,017PF4Q3l4DBUiWoXk4OWT,2814658
2,047WmwIeerHyIUstFAEz5A,3060737
3,04wvWMRKKxK9TGG4IPk32d,90156
4,059bcIhyc2SBwm6sw2AZzd,6447442


Tengo 24400 filas en canciones, de las cuales hay 738 canciones únicas (Dataframe 1, si hago un groupby para que me sume los Streams de las mismas canciones se hace sin problema y tengo un Dataframe 2

Pero ahora no quiero perder las demás columnas haciendo groupby, pero es la forma fácil de hacer las sumas de Streams. Pero entonces quisiera añadir en nueva columna en el dataframe 1 el dato de Streams totales de dicha canción. Lo ideal sería hacer un bucle que mirara si el Track name es el mismo, entonces en una nueva columna añadiría ese dato para esa fila.

In [17]:
# OOOOOtra prueba

data_top200['spotify_id'].map(lambda x: data_top200_sumastreams['Streams'] if data_top200['Track Name'] == data_top200_sumastreams['Track Name'])

SyntaxError: invalid syntax (<ipython-input-17-2269432d3f19>, line 3)

In [18]:
data_top200_sumastreams['spotify_id']

0      00ZBADBKZGwnzGIAA6U9Fb
1      017PF4Q3l4DBUiWoXk4OWT
2      047WmwIeerHyIUstFAEz5A
3      04wvWMRKKxK9TGG4IPk32d
4      059bcIhyc2SBwm6sw2AZzd
                ...          
802    7wCqmkAjudO0oKRASmv8Wg
803    7xX3wvvcZvxGCa06KlgTCp
804    7ytR5pFWmSjzHJIeQkgog4
805    7z9chavxReUpyOZoAkUnUb
806    7zAGi1cps7yYhAajGXlFTH
Name: spotify_id, Length: 807, dtype: object

In [11]:
def conditions(s):
    for songcode in data_top200['spotify_id']:
        if (data_top200['spotify_id'] == data_top200_sumastreams['spotify_id']:
            return data_top200_sumastreams['Streams']

data_top200['NewColumn'] = data_top200.apply(conditions, axis=1)

SyntaxError: invalid syntax (<ipython-input-11-306208444fa4>, line 3)

In [ ]:
def f(row):
    if data_top200['spotify_id'] == data_top200_sumastreams['spotify_id']:
        data_top200_sumastreams['spotify_id'] =
    elif row['A'] > row['B']:
        val = 1
    else:
        val = -1
    return val

In [203]:
#Otra posible forma

data_top200['Streams'] = np.where((data_top200['Track Name']  == data_top200_sumastreams['Track Name']), #Identifies the case to apply to
                           data_top200_sumastreams['Streams'],      #This is the value that is inserted
                           data_top200['Streams'])      #This is the column that is affected

ValueError: operands could not be broadcast together with shapes (24400,) (738,2) (24400,) 

In [ ]:
data_top200.Tr[data_top200.Streams == data_top200_sumastreams['Streams'] = df.col2

In [ ]:
#NUEVOOOO
# https://datascience.stackexchange.com/questions/56668/pandas-change-value-of-a-column-based-another-column-condition


data['column2'] = np.where((data['column1'] > 90)
                           & (data['column2'] ==2), #For rows with column1 > 90
                           data['column3'],      #We place column3 values
                           data['column2'])      #In column two



data['column2'] = np.where((data['column1'] < 30)
                           & (data['column2'] ==2), #Identifies the case to apply to
                           data['column2'],      #This is the value that is inserted
                           data['column2'])      #This is the column that is affected
data['column2'] = np.where((data['column1'] > 90)
                           & (data['column2'] ==2), #For rows with column1 > 90
                           data['column3'],      #We place column3 values
                           data['column2'])      #In column two

In [ ]:
# Ahora habría que: en las canciones que coincidan en el nombre sustituir por dataframe del día por streams totales.
data_top200_sumastreams = data_top200.groupby('Track Name')
data_top200_sumastreams.head()

data_top200['Streams'] = df['Streams'].map(lambda x: data_top200_sumastreams['Streams']
                                           if data_top200['Track Name'] == data_top200_sumastreams['Track Name'])

In [95]:
#Prueba para crear lista con canciones únicas
songs_list = []

for song in data_top200['spotify_id']:
    if song not in songs_list:
        songs_list.append(song)
        

len(songs_list)
#Que se sumen los streams y se quiten columnas

songscode_list = []

for songcode in data_top200['spotify_id']:
    if songcode not in songscode_list:
        songscode_list.append(songcode)
    else: #Está en lista y quiero ahora sumar los strings de ese songcode
        for songcode in data_top200['spotify_id']:
            data_top200['Streams'].sum()

        
#map(lambda x: lasumaaaaa if songcode == data_top200['spotify_id'], songs_list)



807